In [1]:
import nibabel as nib
import numpy as np
import os
from os import path
import pandas as pd
from scipy.spatial import distance_matrix
from sklearn.cluster import DBSCAN
import sys
from zipfile import ZipFile

In [ ]:
from fsedit_notes_helpers import get_diff_data, summarize_edits, get_bm_edits, get_wm_edits, get_cp_edits, get_bfs_edits

In [2]:
subnum = 'CC0142'
typ = 'bm'
basedir='/Users/zeynepenkavi/Downloads/ConteQC'
outdir='/Users/zeynepenkavi/Downloads/ConteQC'

In [3]:
editp = path.join(basedir, 'sub-'+subnum)
uneditp = path.join(basedir, 'sub-'+subnum+'_unedited')
typ_dict = {'bm': 'brainmask', 'wm': 'wm', 'bfs': 'brain.finalsurfs', 'cp': 'control.dat'}
vol = typ_dict[typ]

In [27]:
typ = "wm"
vol = typ_dict[typ]

In [28]:
edit_img = nib.load(path.join(editp,'mri/%s.mgz')%(vol))
edit_data = edit_img.get_fdata()
unedit_img = nib.load(path.join(uneditp,'mri/%s.mgz')%(vol))
unedit_data = unedit_img.get_fdata()
diff_data = unedit_data - edit_data

In [29]:
diffp = path.join(basedir, 'sub-'+subnum+'_diff')
#aff = np.round(unedit_img.affine, 3)
diff_img = nib.MGHImage(diff_data.astype(np.uint8), unedit_img.affine)

In [30]:
nib.save(diff_img, '%s/diff_%s.mgz'%(diffp, vol))

In [33]:
sum(sum(sum(diff_data)))

35235.0

In [ ]:
typ = "wm"
vol = typ_dict[typ]
get_wm_edits(editp, uneditp, vol, subnum)

In [ ]:
tmp = get_diff_data(editp, uneditp, vol, subnum)
cur_diff_data = tmp.query("Action=='add voxel'")

In [ ]:
cur_diff_data

In [ ]:
dm = distance_matrix(cur_diff_data.loc[:,('Sag', 'Axe', 'Cor')], cur_diff_data.loc[:,('Sag', 'Axe', 'Cor')])

In [ ]:
clustering = DBSCAN(eps=50, min_samples=5).fit(dm)

In [ ]:
list(clustering.labels_)

In [ ]:
#cur_diff_data['cluster'] = list(clustering.labels_)
cur_diff_data.insert(0, "cluster", list(clustering.labels_), True)

In [ ]:
cur_diff_data

In [ ]:
summarize_edits(tmp)

In [ ]:
get_bm_edits(editp, uneditp, vol, subnum)

In [ ]:
typ = "cp"
vol = typ_dict[typ]
get_cp_edits(editp, uneditp, vol, subnum)

In [ ]:
typ = "bfs"
vol = typ_dict[typ]
get_bfs_edits(editp, uneditp, vol, subnum)

In [ ]:
fn_dict = {'bm': get_bm_edits, 'wm': get_wm_edits, 'cp': get_cp_edits, 'bfs': get_bfs_edits}
typ = "all"
if typ == 'all':
    out = pd.DataFrame()
    # loop through function lookup dictionary and run all edit extraction functions
    for k,v in fn_dict.items():
        vol = typ_dict[k]
        out = out.append(v(editp=editp, uneditp=uneditp, vol=vol, subnum=subnum))

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
out

In [ ]:
import shutil

In [ ]:
from os import path

In [ ]:
subnum = 'CC0076_core1'
basedir='/Users/zeynepenkavi/Downloads/ConteQC'
uneditp = path.join(basedir, 'sub-'+subnum+'_unedited')
shutil.rmtree(uneditp)

To do still: 

next step i need to do: 
refine previous editsm  

can this be useful for that?  
could at least run all subjects through this and see if i've missed writing down any edits i've made  

how should i go about refining edits?
go over mike's reports and compare for each subject how different i am from dorit/mike
run their edited images (taken from box) through 

so timeline:  
run all subjects through this and see if you've missed recording any edits
move onto refinement of previous edits:
    -download dorit's edits from box and make difference images for 
    -overlay original, my edits difference image, dorit's difference image in freeview and scroll through
move onto editing new subjects